In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# Load the iris dataset
digits = datasets.load_iris()
X = digits.data
y = digits.target

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Define the models
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    KNeighborsClassifier(),
    GaussianNB(),
    GradientBoostingClassifier(),
    MLPClassifier(max_iter=1000),  # Increase max_iter for larger datasets
]

# Initialize a DataFrame to store the results
results = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Train and evaluate each model
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Compute the metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Store the results in the DataFrame
    results.loc[len(results)] = [model.__class__.__name__, accuracy, precision, recall, f1]

# Print the results
print(results)

/Users/vineethsai/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


                        Model  Accuracy  Precision  Recall  F1 Score
0          LogisticRegression       1.0        1.0     1.0       1.0
1      DecisionTreeClassifier       1.0        1.0     1.0       1.0
2      RandomForestClassifier       1.0        1.0     1.0       1.0
3                         SVC       1.0        1.0     1.0       1.0
4        KNeighborsClassifier       1.0        1.0     1.0       1.0
5                  GaussianNB       1.0        1.0     1.0       1.0
6  GradientBoostingClassifier       1.0        1.0     1.0       1.0
7               MLPClassifier       1.0        1.0     1.0       1.0


In [2]:
import pandas as pd
print(pd.__version__)

2.2.1
